# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from dotenv import load_dotenv
import os

# Import GeoApify API key
load_dotenv()
api_key_geo = os.getenv("api_key_geo")

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,puerto ayora,-0.7393,-90.3518,26.47,81,100,2.94,EC
1,1,pemangkat,1.1667,108.9667,26.94,83,100,1.78,ID
2,2,invercargill,-46.4000,168.3500,11.73,93,100,2.70,NZ
3,3,emerald,-23.5333,148.1667,29.53,40,100,2.80,AU
4,4,forster,-32.1667,152.5167,22.34,84,98,8.14,AU


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
firstmap = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    width=1200,
    height=500,
    size="Humidity",
    scale=0.8,
    color="City"
)   

# Display the map
firstmap

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Max Temp"] <= 32) &
    (city_data_df["Cloudiness"] < 30) &
    (city_data_df["Wind Speed"] <= 4.5)
]

# Drop any rows with null values
filtered_df = filtered_df.dropna()

# Display sample data
filtered_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
9,9,san clemente del tuyu,-36.3569,-56.7235,26.16,67,0,4.26,AR
16,16,matthew town,20.9500,-73.6667,25.65,75,0,2.53,BS
17,17,adamstown,-25.0660,-130.1015,25.50,76,9,3.14,PN
37,37,saint-philippe,-21.3585,55.7679,28.27,72,0,4.02,RE
43,43,sao gabriel da cachoeira,-0.1303,-67.0892,27.01,75,14,1.72,BR


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
9,san clemente del tuyu,AR,-36.3569,-56.7235,67,
16,matthew town,BS,20.9500,-73.6667,75,
17,adamstown,PN,-25.0660,-130.1015,76,
37,saint-philippe,RE,-21.3585,55.7679,72,
43,sao gabriel da cachoeira,BR,-0.1303,-67.0892,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel

radius = 20000
params = {
    "categories": "accommodation.hotel",
    "apiKey": api_key_geo,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting Hotel Search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        if name_address["features"]:# Get the name of the first hotel
            hotel_name = name_address["features"][0]["properties"]["name"]
            # Store the hotel name in the DataFrame
            hotel_df.loc[index, "Hotel Name"] = hotel_name
        else:
            # If no hotels found, store "No hotel found"
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    except (KeyError, IndexError, requests.exceptions.RequestException) as e:
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        print(f"Error processing {row['City']}: {str(e)}")
        
    # Log the search results
    print(f"{row['City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting Hotel Search
san clemente del tuyu - nearest hotel: No hotel found
matthew town - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
saint-philippe - nearest hotel: No hotel found
sao gabriel da cachoeira - nearest hotel: No hotel found
punta del este - nearest hotel: No hotel found
kingston - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: No hotel found
mowad - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
dzilam gonzalez - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
chui - nearest hotel: No hotel found
shwebo - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
sao joao da barra - nearest hotel: No hotel found
sao vicente - nearest hotel: No hotel found
isangel - nearest hotel: No hotel found
howard springs - nearest hotel: No hotel found
west bay - nearest hotel: No hotel found
kalmunai - nearest hotel: No hotel found
alice springs - nearest hotel: No h

,City,Country,Lat,Lng,Humidity,Hotel Name
9,san clemente del tuyu,AR,-36.3569,-56.7235,67,No hotel found
16,matthew town,BS,20.9500,-73.6667,75,No hotel found
17,adamstown,PN,-25.0660,-130.1015,76,No hotel found
37,saint-philippe,RE,-21.3585,55.7679,72,No hotel found
43,sao gabriel da cachoeira,BR,-0.1303,-67.0892,75,No hotel found
49,punta del este,UY,-34.9667,-54.9500,54,No hotel found
57,kingston,JM,17.9970,-76.7936,76,No hotel found
77,brisas de zicatela,MX,15.8369,-97.0419,57,No hotel found
110,mowad,IN,21.4667,78.4333,8,No hotel found
127,hermanus,ZA,-34.4187,19.2345,49,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
# NOT SINGLE HOTEL WAS FOUND ACCORDING TO HOTEL_DF